In [15]:
import os
from dotenv import load_dotenv
from typing_extensions import TypedDict
from langchain_anthropic import ChatAnthropic

In [16]:
# Load environment
load_dotenv()
api_key = os.getenv("ANTHROPIC_API_KEY")
llm = ChatAnthropic(model="claude-3-5-sonnet-latest", anthropic_api_key=api_key)

In [17]:
# Shared state definition
class State(TypedDict):
    topic: str
    joke: str
    improved_joke: str
    final_joke: str
    error_message: str

In [18]:
# Shared functions
def generate_joke(state: State):
    """First LLM call to generate initial joke"""
    msg = llm.invoke(f"Write a short joke about {state['topic']}")
    return {"joke": msg.content}

def improve_joke(state: State):
    """Second LLM call to improve the joke"""
    msg = llm.invoke(f"Make this joke funnier by adding wordplay: {state['joke']}")
    return {"improved_joke": msg.content}

def polish_joke(state: State):
    """Third LLM call for final polish"""
    msg = llm.invoke(f"Add a surprising twist to this joke: {state['improved_joke']}")
    return {"final_joke": msg.content}

def check_punchline(state: State):
    """Gate function to check if the joke has a punchline"""
    if "?" in state["joke"] or "!" in state["joke"]:
        return "Pass"
    return "Fail"

# =============================================================================
# WORKFLOW APPROACH - Deterministic, predefined flow
# =============================================================================

In [12]:
def workflow_joke_generator(topic: str) -> State:
    """
    WORKFLOW: Deterministic execution with predefined conditional logic
    
    Key characteristics:
    - Fixed, predictable flow
    - All conditional logic explicitly programmed
    - No dynamic decision making
    - Fail-fast on errors
    - Clear, linear execution path
    """
    
    state = State(topic=topic, joke="", improved_joke="", final_joke="", error_message="")
    
    try:
        # Step 1: Generate initial joke
        print("WORKFLOW Step 1: Generating initial joke...")
        result = generate_joke(state)
        state.update(result)
        
        # Step 2: Quality gate - hardcoded business logic
        print("WORKFLOW Step 2: Checking quality gate...")
        gate_result = check_punchline(state)
        
        if gate_result == "Fail":
            state["error_message"] = "Joke failed quality gate - no punchline detected!"
            return state
        
        # Step 3: Improve joke (only if gate passed)
        print("WORKFLOW Step 3: Improving joke...")
        result = improve_joke(state)
        state.update(result)
        
        # Step 4: Polish joke
        print("WORKFLOW Step 4: Polishing joke...")
        result = polish_joke(state)
        state.update(result)
        
        print("WORKFLOW: Complete!")
        
    except Exception as e:
        state["error_message"] = f"Workflow failed: {str(e)}"
    
    return state


# =============================================================================
# AGENT APPROACH - Dynamic, adaptive behavior
# =============================================================================

In [13]:
from langgraph.graph import StateGraph, END

def create_agent_joke_generator():
    """
    AGENT: Dynamic execution with adaptive decision making
    
    Key characteristics:
    - Can adapt flow based on intermediate results
    - More complex error handling and recovery
    - Can backtrack or take alternative paths
    - Dynamic routing between nodes
    - Self-correcting behavior
    """
    
    # Enhanced functions with agent-like behavior
    def agent_generate_joke(state: State):
        """Agent version with retry logic"""
        try:
            result = generate_joke(state)
            print("AGENT: Generated initial joke")
            return result
        except Exception as e:
            print(f"AGENT: Error generating joke, will retry: {e}")
            return {"joke": "Why did the chicken cross the road? To get to the other side!"}
    
    def agent_improve_joke(state: State):
        """Agent version with quality enhancement"""
        result = improve_joke(state)
        print("AGENT: Enhanced joke with wordplay")
        
        # Agent can make additional decisions based on intermediate results
        if len(result["improved_joke"]) < 50:
            print("AGENT: Joke too short, adding more content...")
            msg = llm.invoke(f"Make this joke longer and more elaborate: {result['improved_joke']}")
            result["improved_joke"] = msg.content
            
        return result
    
    def agent_polish_joke(state: State):
        """Agent version with final optimization"""
        result = polish_joke(state)
        print("AGENT: Applied final polish and twist")
        return result
    
    def agent_quality_gate(state: State):
        """Enhanced gate with multiple criteria"""
        # Agent can use more sophisticated logic
        joke = state["joke"]
        
        # Multiple quality checks
        has_punchline = "?" in joke or "!" in joke
        has_setup = len(joke.split()) > 5
        has_topic = state["topic"].lower() in joke.lower()
        
        if has_punchline and has_setup and has_topic:
            print("AGENT: Quality gate passed - proceeding to improvement")
            return "improve"
        elif has_punchline:
            print("AGENT: Basic quality met - skipping to polish")
            return "polish"
        else:
            print("AGENT: Quality gate failed - regenerating")
            return "regenerate"
    
    def agent_regenerate_joke(state: State):
        """Agent can regenerate if quality is poor"""
        print("AGENT: Regenerating joke with more specific prompt...")
        msg = llm.invoke(f"Write a better, funnier joke about {state['topic']} with a clear setup and punchline")
        return {"joke": msg.content}
    
    # Build the agent graph
    workflow = StateGraph(State)
    
    # Add nodes
    workflow.add_node("generate", agent_generate_joke)
    workflow.add_node("regenerate", agent_regenerate_joke)
    workflow.add_node("improve", agent_improve_joke)
    workflow.add_node("polish", agent_polish_joke)
    
    # Set entry point
    workflow.set_entry_point("generate")
    
    # Add conditional edges (agent decision points)
    workflow.add_conditional_edges(
        "generate",
        agent_quality_gate,
        {
            "improve": "improve",
            "polish": "polish", 
            "regenerate": "regenerate"
        }
    )
    
    # Regenerate can loop back to quality gate
    workflow.add_conditional_edges(
        "regenerate",
        agent_quality_gate,
        {
            "improve": "improve",
            "polish": "polish",
            "regenerate": "regenerate"  # Could potentially loop
        }
    )
    
    # Linear flow after quality gate
    workflow.add_edge("improve", "polish")
    workflow.add_edge("polish", END)
    
    return workflow.compile()

# =============================================================================
# COMPARISON DEMO
# =============================================================================

In [14]:
def compare_approaches(topic: str):
    """Demonstrate the differences between workflow and agent approaches"""
    
    print("=" * 60)
    print("WORKFLOW APPROACH")
    print("=" * 60)
    workflow_result = workflow_joke_generator(topic)
    
    print(f"\nWorkflow Result:")
    if workflow_result.get("error_message"):
        print(f"❌ {workflow_result['error_message']}")
    else:
        print(f"✅ Final joke: {workflow_result.get('final_joke', 'No final joke')}")
    
    print("\n" + "=" * 60)
    print("AGENT APPROACH") 
    print("=" * 60)
    
    agent = create_agent_joke_generator()
    agent_result = agent.invoke({"topic": topic, "joke": "", "improved_joke": "", "final_joke": "", "error_message": ""})
    
    print(f"\nAgent Result:")
    print(f"✅ Final joke: {agent_result.get('final_joke', 'No final joke')}")
    
    print("\n" + "=" * 60)
    print("KEY DIFFERENCES SUMMARY")
    print("=" * 60)
    print("""
    WORKFLOW:
    ✓ Predictable, deterministic execution
    ✓ Simple to understand and debug  
    ✓ Fast execution (no complex routing)
    ✓ Clear failure points
    ✗ Rigid - can't adapt to unexpected situations
    ✗ Fails fast without recovery
    ✗ Limited decision-making capability
    
    AGENT:
    ✓ Adaptive and self-correcting
    ✓ Can handle complex decision trees
    ✓ Robust error recovery
    ✓ Can optimize based on intermediate results
    ✗ More complex to understand and debug
    ✗ Potentially slower (more decision points)
    ✗ Can get stuck in loops if not designed carefully
    
    USE WORKFLOW WHEN:
    - Requirements are well-defined and stable
    - Simple, linear processing is sufficient
    - Predictability and performance are critical
    - You need easy debugging and maintenance
    
    USE AGENT WHEN:
    - Requirements are complex or evolving
    - Need adaptive behavior and error recovery
    - Multiple decision paths are required
    - Self-optimization is valuable
    """)